# Data

## Libraries for Data Processing

In [1]:
%matplotlib inline

String processing and utility libraries

In [1]:
import os
import glob
import string
import re
import imageio

Libraries for functional programming

In [2]:
import operator as op
import itertools as it
from functools import reduce, partial
import toolz as tz
import toolz.curried as c

Libraries for numerical programming

In [3]:
import numpy as np
import pandas as pd
from scipy import (stats, sparse, linalg, 
                   spatial, integrate, optimize, io)

Libraries for plotting and visualization

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

## Working with unstructured text data

### Categorical and one-hot encoding

See examples from S05_Text notebook.

### Term frequency - inverse document frequency (tf-idf)

> In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. The tf-idf value increases proportionally to the number of times a word appears in the document, but is often offset by the frequency of the word in the corpus, which helps to adjust for the fact that some words appear more frequently in general. Nowadays, tf-idf is one of the most popular term-weighting schemes; 83% of text-based recommender systems in the domain of digital libraries use tf-idf.

Source: [Wikipedia](https://en.wikipedia.org/wiki/Tf–idf)

#### Documents 

In [6]:
paths = glob.glob(os.path.join('data', 'Gutenberg', '*.txt') )
print(paths)
names = [os.path.splitext(os.path.split(path)[-1])[0] for path in paths]
names

['data/Gutenberg/carroll-alice.txt', 'data/Gutenberg/shakespeare-macbeth.txt', 'data/Gutenberg/chesterton-ball.txt', 'data/Gutenberg/shakespeare-hamlet.txt', 'data/Gutenberg/blake-poems.txt', 'data/Gutenberg/austen-sense.txt', 'data/Gutenberg/bryant-stories.txt', 'data/Gutenberg/shakespeare-caesar.txt', 'data/Gutenberg/austen-emma.txt', 'data/Gutenberg/milton-paradise.txt', 'data/Gutenberg/edgeworth-parents.txt', 'data/Gutenberg/bible-kjv.txt', 'data/Gutenberg/burgess-busterbrown.txt', 'data/Gutenberg/chesterton-thursday.txt', 'data/Gutenberg/austen-persuasion.txt', 'data/Gutenberg/whitman-leaves.txt', 'data/Gutenberg/melville-moby_dick.txt', 'data/Gutenberg/chesterton-brown.txt']


['carroll-alice',
 'shakespeare-macbeth',
 'chesterton-ball',
 'shakespeare-hamlet',
 'blake-poems',
 'austen-sense',
 'bryant-stories',
 'shakespeare-caesar',
 'austen-emma',
 'milton-paradise',
 'edgeworth-parents',
 'bible-kjv',
 'burgess-busterbrown',
 'chesterton-thursday',
 'austen-persuasion',
 'whitman-leaves',
 'melville-moby_dick',
 'chesterton-brown']

In [7]:
N = len(names)

#### Simple processing to find words in each document

Standard Python idiom.

In [8]:
doc_terms = []
for path in paths:
    with open(path, encoding='latin-1') as f:
        text = f.read()
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation))
        words = text.split()
        doc_terms.append(words)

Using a functional idiom

In [9]:
doc_terms = tz.pipe(
    paths,
    c.map(partial(open, encoding='latin-1')),
    c.map(lambda x: x.read()),
    c.map(lambda x: x.lower()),
    c.map(lambda x: x.translate(str.maketrans('', '', string.punctuation))),
    c.map(lambda x: x.split()),
    list
)

#### Term frequency

This is just a word count for each document.

In [10]:
tf = {name: tz.frequencies(doc) 
      for name, doc in zip(names, doc_terms)}

#### Document frequency

This is how many documents a term appears in.

In [11]:
df = tz.frequencies(tz.concat(d.keys() for d in tf.values()))

#### Inverse document frequency 

This weights each term by the inverse frequency of its appearance across different documents, and reduces the important of common words like "the".

In [12]:
idf = {term: np.log((1 + N)/(1 + count)) for term, count in df.items()}

#### Term frequency - inverse document frequency

This is just the product of tf and idf, and a measure of the importance of each term in a document.

In [13]:
terms = list(tz.unique(tz.concat(doc_terms)))

In [14]:
tf_idf = {}
for name, doc in tf.items():
    d = {}
    for term in terms:
        d[term] = doc.get(term, 0) * idf[term]
    tf_idf[name] = d

#### Convert to a structured data type

In [15]:
tf_idf = pd.DataFrame(tf_idf)

In [16]:
tf_idf.iloc[:5, :5]

,austen-emma,austen-persuasion,austen-sense,bible-kjv,blake-poems
00,0.0,0.0,0.0,0.0,0.0
00021053,0.0,0.0,0.0,0.0,0.0
00081429,0.0,0.0,0.0,0.0,0.0
00482129,0.0,0.0,0.0,0.0,0.0
01,0.0,0.0,0.0,0.0,0.0


#### Distinctive words in each document

These are words that appear frequently in that document but not in others.

In [17]:
pd.DataFrame({doc: series.sort_values(ascending=False).index[:5] 
              for doc, series in tf_idf.items()}).T

,0,1,2,3,4
austen-emma,emma,harriet,weston,knightley,elton
austen-persuasion,elliot,anne,wentworth,musgrove,russell
austen-sense,elinor,marianne,dashwood,jennings,mrs
bible-kjv,unto,israel,saith,thee,thou
blake-poems,thel,weep,thee,lyca,vales
bryant-stories,jackal,margery,brahmin,nightingale,epaminondas
burgess-busterbrown,buster,browns,joe,blacky,billy
carroll-alice,alice,gryphon,dormouse,duchess,hatter
chesterton-ball,turnbull,macian,evan,turnbulls,have
chesterton-brown,flambeau,boulnois,muscari,brown,fanshaw


#### Why is "macb" the top hit for Macbeth? 

In [18]:
tf['shakespeare-macbeth']['macb']

137

In [19]:
text = open('data/gutenberg/shakespeare-macbeth.txt', encoding='latin-1').read()
re.findall('macb[^e].*\n', text, re.IGNORECASE)[:10]

FileNotFoundError: [Errno 2] No such file or directory: 'data/gutenberg/shakespeare-macbeth.txt'

### Image data

In [ ]:
paths = glob.glob(os.path.join('data', 'POKEMON', '*.png') )[:3]

imgs = np.array([imageio.imread(path) for path in paths])

In [ ]:
imgs.shape

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(10,3))
for ax, img in zip(axes, imgs):
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])

## Using `pandas` for data munging

In [8]:
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv"

In [9]:
tips = pd.read_csv(url)

### Inspecting a data frame

In [ ]:
tips.shape

In [ ]:
tips.dtypes

In [ ]:
tips.columns

In [ ]:
tips.index

In [ ]:
tips.describe()

In [ ]:
tips.head(3)

In [ ]:
tips.tail(3)

In [ ]:
tips.sample(3)

### Series

A column is a pandas Series object. It behaves like an indexed vector.

In [ ]:
tips['sex'].head()

Alternative way to get Series when column name meets requirements for Python variable (i.e. no spaces or punctuation)

In [ ]:
tips.sex.head()

### Series types

#### String operations

In [ ]:
tips.sex.str.lower().str[0].head()

#### Categorical data types

In [ ]:
tips.day.unique()

In [ ]:
tips['day'] = tips.day.astype('category')

In [ ]:
tips.day.head(3)

In [ ]:
tips.day.cat.reorder_categories(['Thur', 'Fri', 'Sat', 'Sun'], ordered=True, inplace=True)

In [ ]:
tips.day.head(3)

#### Datetime operations

In [3]:
import pandas_datareader as pdr

data_source = 'google'
start_date = '2010-01-01'
end_date = '2016-12-31'

data = pdr.get_data_yahoo('MSFT', start_date, end_date)

In [4]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,30.620001,31.10,30.590000,30.950001,25.031292,38409100
2010-01-05,30.850000,31.10,30.639999,30.959999,25.039379,49749600
2010-01-06,30.879999,31.08,30.520000,30.770000,24.885712,58182400


In [6]:
dates = data.index.get_level_values(0)

In [7]:
dates

DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08', '2010-01-11', '2010-01-12', '2010-01-13',
               '2010-01-14', '2010-01-15',
               ...
               '2016-12-16', '2016-12-19', '2016-12-20', '2016-12-21',
               '2016-12-22', '2016-12-23', '2016-12-27', '2016-12-28',
               '2016-12-29', '2016-12-30'],
              dtype='datetime64[ns]', name='Date', length=1762, freq=None)

In [ ]:
dates[:3]

In [ ]:
list(it.islice(zip(dates.year, 
                    dates.month,
                    dates.day), 6))

In [ ]:
msft = data.loc['MSFT']

In [ ]:
msft.plot(y='High')
pass

In [ ]:
msft.loc['2016-12-01':'2016-12-31', ('Low', 'High')].plot(
    linestyle='dashed', marker='+')
pass

### Indexing

In [ ]:
tips[0:2]

In [ ]:
tips.loc[0:2]

In [ ]:
tips.iloc[0:2]

In [ ]:
tips.iloc[0:3, [2,3,4]]

In [ ]:
tips.loc[0:2, ['tip', 'sex', 'size']]

In [ ]:
tips.loc[0:2, 'tip':'day']

#### Boolean indexing

In [ ]:
tips[tips.sex == 'Male'].head(3)

In [ ]:
tips[(tips.sex == 'Male') & (tips.time != 'Dinner')].head(3)

### Special selection

In [8]:
df = pd.DataFrame({
    'a': [1,2,None,4,1], 
    'b': [3,3,None,None,3],
    'c': [1,2,None,4,1]})

In [9]:
df

,a,b,c
0,1.0,3.0,1.0
1,2.0,3.0,2.0
2,NaN,NaN,NaN
3,4.0,NaN,4.0
4,1.0,3.0,1.0


In [10]:
df.dropna()

,a,b,c
0,1.0,3.0,1.0
1,2.0,3.0,2.0
4,1.0,3.0,1.0


In [11]:
df.dropna(how='all',axis=0)
#df.dropna(how='all',axis=1)
#df.dropna(how='all')

,a,b,c
0,1.0,3.0,1.0
1,2.0,3.0,2.0
3,4.0,NaN,4.0
4,1.0,3.0,1.0


In [ ]:
df.drop_duplicates()

In [ ]:
df.drop_duplicates(keep='last')

### Selecting by label

In [ ]:
tips.filter(regex='^s.*').head(3)
#all the column names that start from 's'
#default is column filter

### Sorting

In [ ]:
tips.sort_values(['size', 'tip'], ascending=[True, False]).head(3)

Note that ordered categorical values are sorted appropriately.

In [ ]:
tips.sort_values(['day']).head(3)

### Rearrange columns

In [ ]:
tips = tips[tips.columns.sort_values()]
tips.head(3)

In [10]:
tips = tips.filter('total_bill	tip	sex	smoker	day	time	size'.split())
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


In [6]:
'total_bill tip sex smoker day time size'.split()

['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

### Transforms

In [ ]:
tips['day_type'] = 'Weekday'
tips.loc[tips.day.isin(['Sat', 'Sun']), 'day_type'] = 'Weekend'
tips.head(3)

In [ ]:
tips['cost'] = tips['total_bill'] + tips['tip']
tips.head(3)

In [ ]:
tips.assign(log1p_cost = np.log1p(tips.cost)).head(3)

In [ ]:
tips.replace({
    'sex': dict(Femals='F', Male='M'),
    'day': dict(Thur=4, Fri=5, Sat=6, Sun=7)}).head(3)

In [1]:
{'sex': dict(Femals='F', Male='M'),
    'day': dict(Thur=4, Fri=5, Sat=6, Sun=7)}

{'day': {'Fri': 5, 'Sat': 6, 'Sun': 7, 'Thur': 4},
 'sex': {'Femals': 'F', 'Male': 'M'}}

In [ ]:
tips.rename({'sex': 'gender'}, axis=1).head(3)

#### Transforming missing values

In [ ]:
df

In [ ]:
df.fillna(0)

In [ ]:
df.fillna(df.mean())

In [ ]:
df.fillna(method='ffill')
# fill in the former value

In [ ]:
df.fillna(method='bfill')
# fill in the back value

### Summaries

In [ ]:
tips.mean()

In [ ]:
tips.std()

In [ ]:
tips.count()

### Grouping

In [10]:
tips.groupby(['sex', 'day', 'time']).mean()

total_bill       tip      size
sex    day  time                                  
Female Fri  Dinner   14.310000  2.810000  2.000000
            Lunch    13.940000  2.745000  2.250000
       Sat  Dinner   19.680357  2.801786  2.250000
       Sun  Dinner   19.872222  3.367222  2.944444
       Thur Dinner   18.780000  3.000000  2.000000
            Lunch    16.648710  2.561935  2.483871
Male   Fri  Dinner   23.487143  3.032857  2.285714
            Lunch    11.386667  1.900000  1.666667
       Sat  Dinner   20.802542  3.083898  2.644068
       Sun  Dinner   21.887241  3.220345  2.810345
       Thur Lunch    18.714667  2.980333  2.433333

In [11]:
tips.groupby(['sex']).agg(['mean', 'std', 'min', 'max'])

total_bill                              tip                       \
             mean       std   min    max      mean       std  min   max   
sex                                                                       
Female  18.056897  8.009209  3.07  44.30  2.833448  1.159495  1.0   6.5   
Male    20.744076  9.246469  7.25  50.81  3.089618  1.489102  1.0  10.0   

            size                    
            mean       std min max  
sex                                 
Female  2.459770  0.937644   1   6  
Male    2.630573  0.955997   1   6

### Working with hierarchical indexes

In [12]:
df = tips.groupby(['sex', 'day', 'time']).mean()
df.head()

total_bill       tip      size
sex    day  time                                  
Female Fri  Dinner   14.310000  2.810000  2.000000
            Lunch    13.940000  2.745000  2.250000
       Sat  Dinner   19.680357  2.801786  2.250000
       Sun  Dinner   19.872222  3.367222  2.944444
       Thur Dinner   18.780000  3.000000  2.000000

In [13]:
df.index

MultiIndex(levels=[['Female', 'Male'], ['Fri', 'Sat', 'Sun', 'Thur'], ['Dinner', 'Lunch']],
           labels=[[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], [0, 0, 1, 2, 3, 3, 0, 0, 1, 2, 3], [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1]],
           names=['sex', 'day', 'time'])

In [14]:
df.columns

Index(['total_bill', 'tip', 'size'], dtype='object')

In [15]:
df.loc[('Female')]

total_bill       tip      size
day  time                                  
Fri  Dinner   14.310000  2.810000  2.000000
     Lunch    13.940000  2.745000  2.250000
Sat  Dinner   19.680357  2.801786  2.250000
Sun  Dinner   19.872222  3.367222  2.944444
Thur Dinner   18.780000  3.000000  2.000000
     Lunch    16.648710  2.561935  2.483871

In [ ]:
df.loc[('Female', 'Sat')]

In [ ]:
df.loc[('Female', slice(None), 'Dinner')]  ###????
#'slice(None)' gives everything

### Stacking and unstacking

In [ ]:
tips.head(3)

In [ ]:
df.unstack()['tip']

In [ ]:
df.unstack(level=0)['tip']

In [ ]:
df.unstack(level=[1,2])['tip']

In [ ]:
df.unstack(level=[1,2])['tip'].stack(level=0)

#### Swapping levels

In [ ]:
df.swaplevel(1,2)

### Resetting index

If you'd rather not deal with hierarchical indexes, use `reset_index`.

In [ ]:
df.reset_index()

### Reshaping

In [ ]:
df1 = df.reset_index()

In [ ]:
df1.head(3)

#### Transpose

In [ ]:
df1.T

#### Melt (gather)

In [ ]:
pd.melt(df1, id_vars=['sex', 'day', 'time', 'size']).head(10)

### Pivot table

A pivot table is like a group_by operation on both the index (rows) and columns.

In [ ]:
pd.pivot_table(df1, values=['total_bill', 'tip'],
               index=['sex', 'day'], 
               columns='time',
               aggfunc='mean')

### Joining data frames

In [ ]:
tips1 = tips[0:5]
tips1

In [ ]:
tips2 = tips[3:8]
tips2

#### Simple concatenation

In [ ]:
pd.concat([tips1, tips2])

### Joining columns

Merge uses all common columns to combine. It is very flexible - see help(pd.merge).

In [ ]:
pd.merge(tips1, tips2)

In [ ]:
pd.merge(tips1, tips2, how='left')

In [ ]:
pd.merge(tips1, tips2, how='right')

In [ ]:
pd.merge(tips1, tips2, how='outer')

## Visualizing data

In [ ]:
tips.head()

In [ ]:
sns.set_context('notebook', font_scale=1.3)
g = sns.factorplot(x='sex', y='tip',
                   col='time', row='smoker',
                   data=tips, kind='swarm')
pass

In [ ]:
sns.set_context('notebook', font_scale=1.3)
g = sns.factorplot(x='day', y='tip', hue='sex',
                   col='time', row='smoker',
                   data=tips, kind='violin', 
                   margin_titles=True)
pass

In [ ]:
sns.lmplot(x='total_bill', y='tip', hue='sex', 
           col='time', row='smoker', 
           data=tips, palette='dark')
pass